In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, BatchNormalization
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier

In [3]:
data = pd.read_csv('weatherHistory.csv')

data['Date'] = pd.to_datetime(data['Formatted Date'], utc=True, errors='coerce')


data = data.dropna(subset=['Date'])


data['Day'] = data['Date'].dt.day
data['Month'] = data['Date'].dt.month
data['Year'] = data['Date'].dt.year


data['Rain'] = np.random.randint(0, 2, size=len(data))


X = data[['Temperature (C)', 'Humidity', 'Wind Speed (km/h)', 'Pressure (millibars)']]
y = data['Rain']


scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  
])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
optimizer = Adam(learning_rate=0.0001, clipvalue=1.0)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
def predict_rain(model, scaler, input_data):
    if len(input_data) == 3:
        last_weather_data = X.iloc[-1][['Temperature (C)', 'Humidity', 'Wind Speed (km/h)', 'Pressure (millibars)']].values
        input_data = np.concatenate([last_weather_data])

    # Normalize ข้อมูล
    input_data_scaled = scaler.transform([input_data])

    # พยากรณ์ด้วยโมเดล
    predict = model.predict(input_data_scaled)
    probability = predict[0][0] * 100  # Convert to percentage
    
    # Return ผลลัพธ์
    if probability > 50:
        return probability, "Rain is likely"
    else:
        return probability, "No Rain expected"

print("ตัวอย่างวันที่ 06/09/2024 ใส่ตามที่ขึ้นไม่ใช่ใส่หมดทีเดียว")
iD = int(input("Day: "))
iM = int(input("Month: "))
iY = int(input("Year: "))
roun = int(input("Number of training epochs: "))
batch_siz = int(input("Batch size: "))
train_round = int(input("ใส่รอบในการ train ซํ้าเพื่อหาค่าเฉลี่ย แนะนำคือ 3 รอบ ทั่งนี่ หากไม่ต้องการให้ใส่ 1"))
train_iterations = train_round  # กำหนดจำนวนรอบการ train

# เก็บค่าเฉลี่ย
rain_probabilities = []

for i in range(train_iterations):
    print(f"Training iteration {i+1}/{train_iterations}")
    
    model.fit(X_train, y_train, epochs=roun, batch_size=batch_siz, verbose=0)

    # Train เสร็จบันทึก model
    model.save("model.h5")

    # พยากรณ์ด้วยข้อมูลวันสุดท้าย
    X_last_day = X.iloc[-1].values
    probability_last_day, _ = predict_rain(model, scaler, X_last_day[:3])
    
    # เพิ่มค่า rain_probabilities
    rain_probabilities.append(probability_last_day)


average_probability = sum(rain_probabilities) / len(rain_probabilities)

print("----------------------------------------------------------------")
print(f"ค่าเฉลี่ยความน่าจะเป็นในการเกิดฝนหลังจาก {train_iterations} รอบ: {average_probability:.2f}%")
print("================================================================")


result_custom_date = predict_rain(model, scaler, np.array([iD, iM, iY]))
print("===========================OUTPUT===============================")
print(f"สำหรับวันที่ {iD}/{iM}/{iY}: {result_custom_date[1]} (ความน่าจะเป็น: {result_custom_date[0]:.2f}%)")
print("================================================================")


ตัวอย่างวันที่ 06/09/2024 ใส่ตามที่ขึ้นไม่ใช่ใส่หมดทีเดียว
Training iteration 1/3


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Training iteration 2/3


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Training iteration 3/3


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


----------------------------------------------------------------
ค่าเฉลี่ยความน่าจะเป็นในการเกิดฝนหลังจาก 3 รอบ: 51.02%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
===========================OUTPUT===============================
สำหรับวันที่ 13/9/2024: Rain is likely (ความน่าจะเป็น: 51.15%)
